# Interpretando los Resultados de la Caja Negra

Una vez que hemos entrenado una caja negra para hacer predicciones podemos querer saber mas a cerca de como esta caja negra toma esas decisiones aun sin entender completamente que es lo que sucede dentro de ella. Para ello veremos varias tecnicas que nos pueden ayudar a interpretar los resultados.

## Tecnicas de Caja Negra

Estas tecnicas sirven para interpretar los resultados aun sin entender absolutamente nada de lo que pasa adentro de las cajas negras ni cual fue el algoritmo utilizado para entrenarlas.

Existen varias tecnicas y librerias para hacer esto, a continuacion se mencionan algunas:
  - [Sklearn Inspection](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.inspection)
  - [Shap](https://shap.readthedocs.io/en/latest/index.html)
  - [ELI5](https://eli5.readthedocs.io/en/latest/)
  - [imodels](https://github.com/csinva/imodels)
  
En esta unidad veremos la tecnica mas sencilla llamada Feature Permutation y que consiste en seleccionar una columna especifica de mi dataset, mezclar sus filas y calcular el error del modelo. Si esta mezcla de las filas ha modificado mucho su poder de prediccion entonces quiere decir que esa columna es importante, si de lo contrario no ha modificado su poder de prediccion quiere decir que esa columna no ha aportado mucho a la prediccion. De esta manera podemos saber cuales han sido las columnas que el modelo ha considerado mas importantes a la hora de hacer la prediccion.

Veamos ahora un ejemplo con el modelo complejo que hemos obtenido en la unidad anterior:

In [ ]:
!pip install "flaml[automl]"

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('https://raw.githubusercontent.com/amiune/freecodingtour/main/cursos/espanol/datascience/data/diabetes/diabetes_train_procesado.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/amiune/freecodingtour/main/cursos/espanol/datascience/data/diabetes/diabetes_test_procesado.csv')

In [ ]:
X_train = df_train.loc[:, df_train.columns != "diabetes"]
y_train = df_train.loc[:, "diabetes"]

In [ ]:
from flaml import AutoML
automl = AutoML()
automl.fit(X_train, y_train, task="classification", time_budget=10)

In [ ]:
automl.best_estimator

In [ ]:
!pip install eli5

In [ ]:
X_test = df_test.loc[:, df_test.columns != "diabetes"]
y_test = df_test.loc[:, "diabetes"]

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(automl, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

Aqui vemos la lista de las columnas ordenadas por orden de importancia de mayor (verde) a menor (rojo)

## Tecnicas de Analisis de Modelos Simples

Cuando elegimos modelos simples como:
- Regresion Lineal
- Arbol de Decision

Entonces es mucho mas simple entender que es lo que esta haciendo el modelo ya que estos algoritmos son simples de entender. Para ver mas en detalle como funcionan ir a la seccion de material avanzado "Espiando dentro de las cajas negra."

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/datascience/datascience.html)